## Generate html file with map and bubbles for points of interest

In [1]:
import numpy as np
import pandas as pd
import folium

In [2]:
# Create folium map object
def create_map(df):
    lon_mean = df['longitude'].mean()
    lat_mean = df['latitude'].mean()
    return folium.Map(location=[lat_mean, lon_mean], tiles="OpenStreetMap", zoom_start=11)

In [3]:
# Add bubbles to map object

def add_to_map(m, # map object
               df, # feed data frame
#                column, # column with object name, e.g. shop chain
#                criteria_str, # object name to include/exclude, e.g. 'Magnit'
#                criteria = 'include', # include/exclude based on object name
               bubble_r = 100, # bubble radius in meters
#                 bubble appearance 
               outline_color = 'red',
               fill=True,
               fill_color = 'red',
               opacity = 0.7):
    
        
#     add bubbles into map object one by one
    for i in range(df.shape[0]):
       folium.Circle(
          location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
#           popup=filter_df.iloc[i][column],
          radius=bubble_r,
          color=outline_color,
          fill=fill,
          fill_color=fill_color,
          opacity = opacity
       ).add_to(m)

In [4]:
lonm = 62000. 
latm = 111111.
def rast(lat1,lon1, lat2, lon2, lonm = lonm, latm = latm):
    return(((latm* (lat2- lat1))**2 + (lonm*(lon2-lon1))**2)**0.5)

In [5]:
# Save map object as html
# m - map object
# html_name - html name string
def save_map(m, html_name):
    m.save(html_name + '.html')

In [6]:
dots = pd.read_excel('./data/for_peresekator.xlsx')

In [7]:
dots.columns = ['clusters', 'latitude', 'longitude']
clusters_df = dots.groupby('clusters')[['latitude', 'longitude']].mean().reset_index(drop=True)
clusters_df['clusters'] = clusters_df.index.values

In [8]:
filtered_0 = dots[dots['clusters']==0].reset_index(drop=True)
filtered_1 = dots[dots['clusters']==1].reset_index(drop=True)
filtered_2 = dots[dots['clusters']==2].reset_index(drop=True)
filtered_0['radius'] = (abs(filtered_0['latitude'] - clusters_df.latitude[0]) + \
                    abs(filtered_0['longitude'] - clusters_df.longitude[0]))**0.5
filtered_1['radius'] = (abs(filtered_1['latitude'] - clusters_df.latitude[1]) + \
                    abs(filtered_1['longitude'] - clusters_df.longitude[1]))**0.5
filtered_2['radius'] = (abs(filtered_2['latitude'] - clusters_df.latitude[2]) + \
                    abs(filtered_2['longitude'] - clusters_df.longitude[2]))**0.5

In [9]:
mapa = create_map(dots)

In [10]:
# 2_cluster
folium.Circle(
  location=[clusters_df.iloc[2]['latitude'], clusters_df.iloc[2]['longitude']],

   radius = rast(clusters_df.iloc[2]['latitude'],clusters_df.iloc[2]['longitude'],\
                 filtered_2.at[np.argmax(filtered_2.radius), 'latitude'],\
                 filtered_2.at[np.argmax(filtered_2.radius), 'longitude']) +100,
   color = 'red',
   outline_color = 'red',
   fill=True,
   fill_color = 'red',
   opacity = 0.7
).add_to(mapa);

In [11]:
# 1_cluster
folium.Circle(
  location=[clusters_df.iloc[1]['latitude'], clusters_df.iloc[1]['longitude']],

   radius = rast(clusters_df.iloc[1]['latitude'],clusters_df.iloc[1]['longitude'],\
                 filtered_1.at[np.argmax(filtered_1.radius), 'latitude'],\
                 filtered_1.at[np.argmax(filtered_1.radius), 'longitude']) +100,
   color = 'red',
   outline_color = 'red',
   fill=True,
   fill_color = 'red',
   opacity = 0.7
).add_to(mapa);

In [12]:
# 0_cluster
folium.Circle(
  location=[clusters_df.iloc[0]['latitude'], clusters_df.iloc[0]['longitude']],

   radius = rast(clusters_df.iloc[0]['latitude'],clusters_df.iloc[0]['longitude'],\
                 filtered_0.at[np.argmax(filtered_0.radius), 'latitude'],\
                 filtered_0.at[np.argmax(filtered_0.radius), 'longitude'])+100,
   color = 'red',
   outline_color = 'red',
   fill=True,
   fill_color = 'red',
   opacity = 0.7
).add_to(mapa);

In [13]:
#dots from 1_cluster
add_to_map(mapa, # map object
               filtered_1, # feed data frame
               bubble_r = 10, # bubble radius in meters
#                 bubble appearance 
               outline_color = 'blue',
               fill=True,
               fill_color = 'blue',
               opacity = 1)

In [14]:
#dots from 2_cluster
add_to_map(mapa, # map object
               filtered_2, # feed data frame
               bubble_r = 10, # bubble radius in meters
#                 bubble appearance 
               outline_color = 'green',
               fill=True,
               fill_color = 'green',
               opacity = 1)

In [15]:
#dots fro 0_cluster
add_to_map(mapa, # map object
               filtered_0, # feed data frame
               bubble_r = 10, # bubble radius in meters
#                 bubble appearance 
               outline_color = 'black',
               fill=True,
               fill_color = 'black',
               opacity = 1)

In [16]:
# radius_0
folium.ColorLine([[clusters_df.latitude[0], clusters_df.longitude[0]],\
                  [filtered_0.at[filtered_0.radius.argmax(), 'latitude'],\
                  filtered_0.at[filtered_0.radius.argmax(), 'longitude']]],\
                  colors=[0,1,2], colormap=['yellow', 'blue'], weight=3 ).add_to(mapa)
#radius_1
folium.ColorLine([[clusters_df.latitude[1], clusters_df.longitude[1]],\
                  [filtered_1.at[filtered_1.radius.argmax(), 'latitude'],\
                  filtered_1.at[filtered_1.radius.argmax(), 'longitude']]],\
                  colors=[0,1,2], colormap=['yellow', 'yellow'], weight=3 ).add_to(mapa)
#radius_2
folium.ColorLine([[clusters_df.latitude[2], clusters_df.longitude[2]],\
                  [filtered_2.at[filtered_2.radius.argmax(), 'latitude'],\
                  filtered_2.at[filtered_2.radius.argmax(), 'longitude']]],\
                  colors=[0,1,2], colormap=['yellow', 'yellow'], weight=3 ).add_to(mapa);

In [17]:
mapa

In [18]:
save_map(mapa, 'Clusters')